# Welcome! This is Antonio Cervantes and Christian Pedrigal's Machine Learning Final Project!

## Implementation of an Activity Detection System

#### Fall 2021 | Feruza Amirkulova ME 297-01 | Introduction to Optimization and Machine Learning |San Jose State University

# Imports

In [1]:
import os
import numpy as np
%matplotlib widget
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Nadam
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, train_test_split

# Inputs
Data file should be a .csv file in the format of

<font color=red>feature1, feature2, feature3, ..., featureN, activity_label

Features can be from any number of XYZ axes and from either or all sensors decided by engineers, but be sure to note which order they are placed in, and the order of the sensor type.

#### Read Data Table

In [2]:
datafile = read_csv(r'.\data\finalDateTable.csv', header=0)
dataset = datafile.values

#### View Data Table

In [3]:
datafile

,AngFreqX,AngFreqY,AngFreqZ,AmpX,AmpY,AmpZ,MedJerkX,MedJerkY,MedJerkZ,LabelCol
0,2.344570,4.769465,3.453826,43.499893,29.784348,29.201104,36.246550,37.524702,38.218704,Bike
1,7.032146,14.292519,10.349982,6.386593,45.329783,2.792721,40.906822,40.193130,38.367857,Bike
2,7.032146,23.815572,3.453826,2.819911,52.104106,3.694607,48.761701,48.746860,48.556502,Bike
3,2.344570,23.815572,17.246137,2.151884,58.046007,2.368276,43.856071,43.128339,41.641957,Bike
4,4.688358,4.769465,3.453826,3.354227,50.679265,0.996908,39.248316,39.740246,39.716234,Bike
...,...,...,...,...,...,...,...,...,...,...
2477,11.161678,22.681875,16.425166,97.280707,31.785980,40.308343,53.185629,54.971847,52.154658,Walk
2478,11.719723,23.815572,17.246137,97.368660,31.141359,39.995816,42.071573,43.047521,42.430035,Walk
2479,11.719723,23.815572,17.246137,97.547687,32.242386,41.831902,49.061065,49.725893,49.762876,Walk
2480,11.719723,23.815572,17.246137,97.723831,31.206160,40.676614,53.100032,48.206290,47.133008,Walk


#### Parse the dataset and labels

In [15]:
# Set data object as all row and all columns up to the last one
data = np.array(dataset[:,:-1]).astype(float)

# Set labels object as all row of the last column
labels = dataset[:,-1]

#### Encode the labels

In [16]:
# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_labels = encoder.transform(labels)

# Define the number of classes in the dataset
classes = encoded_labels[-1] + 1

#### Define the number of features and classes

In [17]:
# Define the number of features in the dataset
features = len(data[0,:])

# Define the number of classes in the dataset
classes = encoded_labels[-1] + 1

# Training

#### Define the Nueral Network

In [18]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(27, input_dim = features, activation='relu'))
	model.add(Dense(classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

#### Create a Keras Classifier object

In [19]:
# Create Estimator
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=1, verbose=0)

#### Create a KFold object

In [20]:
kfold = KFold(n_splits=4, shuffle=True)

#### Create a dummy labels for evaluation

In [21]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_labels = to_categorical(encoded_labels)

# Evaulation

#### Use Cross Valuation Score to evaluate the accuracy of the model

In [22]:
results = cross_val_score(estimator, data, dummy_labels, cv=kfold, error_score='raise')
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 87.83% (1.50%)
